In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pandas import read_csv
from matplotlib import pyplot
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import pacf 
from statsmodels.tsa.stattools import acf
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from numpy import asarray
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import GradientBoostingRegressor
from matplotlib import pyplot
import pandas as pd
import xgboost as xgb
import sklearn

In [ ]:
sklearn. __version__ 

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = (1 if type(data) is list else data.shape[1])
    df = DataFrame(data)
    cols = list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
    # put it all together
    agg = concat(cols, axis=1)
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg.values
# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
    return data[:-n_test, :], data[-n_test:, :]


#### get the list of all input files

In [ ]:
import os
file_list = list(l for l in os.listdir() if ('covid_ai_model_input_data' in l) and ('test' not in l) )
file_list

#### Predict D1_COVID_NEW_ADM_CNT Univariate

In [ ]:
from datetime import datetime
from datetime import datetime as dt, timedelta
# import forestci as fci
# features_to_add =['POSITIVITY_RATE_MEAN', 'COVID_MED_SURG_NO_HFNC_CNT_M', 'COVID_MED_SURG_CNT_M',	'COVID_NEW_ADM_MEAN',\
#                   'COVID_POS_PT_CNT_EVER',	'COVID_PT_CNT_M',	'COVID_PT_CNT_MIDNIGHT_MEAN','INDX_UCDH_POSITIVITY_RATE',\
#                  'INDX_UCDH_POS_PT_CNT', 'COVID_NEW_ADM_CNT', 'INDX_POS_PT_ALL_ADM_CNT', 'COVID_DISCHARGE_MEAN',	'COVID_ICU_NO_VENT_CNT_M',\
#                   'COVID_ACTIVE_DNR_CNT_M',	'OUTREACH_POSITIVITY_MEAN', 'TESTED_PT_CNT_MEAN', 'INDX_POS_PT_NEW_ADM_CNT', \
#                   'COVID_DISCHARGE_CNT']
# # features_to_add =["POSITIVITY_RATE_MEAN"]
feature_to_predict = ['D1_COVID_NEW_ADM_CNT']
lag_new_D1 =2
import os
file_list = list(l for l in os.listdir() if ('covid_ai_model_input_data' in l) and ('test' not in l) )
for f in file_list:
    df = pd.read_csv(f)
    df['SOURCE_DATA_DATE'] = df['SOURCE_DATA_DATE'].astype('datetime64[ns]')

    # # transform the time series data into supervised learning
    ## ------------------------------------------------
    index_date_sub1 = df['SOURCE_DATA_DATE'].iloc[-2]
    df_validation = df[df['SOURCE_DATA_DATE'] <= index_date_sub1 ].reset_index(drop= True)
    data_multi =df_validation[feature_to_predict].values
    #data =(df_validation['COVID_MED_SURG_NO_HFNC_PCT_M'].values)/100

    # data = data.reshape(-1,1)
    data_ML = pd.DataFrame()
    for r in range(data_multi.shape[1]):
        data_current = data_multi[:,r].reshape(-1,1)
    #     data = series_to_supervised(data_current, n_in=lag_D2+1)
        data = series_to_supervised(data_current, n_in=lag_new_D1)
    #     if r == data_multi.shape[1]-1:
    #         data = data[:,1:]
    #     if r != data_test.shape[1]-1:
    #         data = data[-lag-2:,-1] ## updated to make 5 (lag of 3 +2)past days 
    #     else:
    #         data = data[-lag:,-1]

        data_ML = pd.concat([data_ML, pd.DataFrame(data)], axis= 1)

    train_current = data_ML.values
    # split into input and output columns
    # trainX, trainy = train_current[:-1, :-1], train_current[:-1, -1]
    trainX, trainy = train_current[:, :-1], train_current[:, -1]
    # # fit model

    # tscv = TimeSeriesSplit(test_size = 1, n_splits= 5,gap =1)
    tscv = TimeSeriesSplit(test_size = 1, n_splits= 30)  

    alpha = 0.95
    param_grid = {'n_estimators':[50, 100, 200, 300, 500],'learning_rate':[0.01, 0.1,1],\
                                   'max_depth':[1,2,4]}
    clf = GradientBoostingRegressor(loss='quantile', alpha=alpha, random_state= 1)
    regression_model = GridSearchCV(clf, param_grid= param_grid , \
                                      cv = tscv, scoring= ['neg_mean_squared_error'],refit= 'neg_mean_squared_error', n_jobs=-1)
    # # regression_model = GridSearchCV(clf, param_grid= param_grid , \
    # #                                 cv = tscv, scoring= ['neg_mean_squared_error'],refit= 'neg_mean_squared_error', n_jobs=-1)
    # #     regression_model = GridSearchCV(clf, param_grid= param_grid ,cv = tscv, n_jobs=-1)
    regression_model.fit(trainX, trainy)

    row = trainy[-lag_new_D1:]
    # Make the prediction on the meshed x-axis
    y_upper = regression_model.predict([row])
    print('Upper quantile: {}'.format(y_upper))

    clf.set_params(alpha=1.0 - alpha)
    regression_model.fit(trainX, trainy)

    # Make the prediction on the meshed x-axis
    y_lower = regression_model.predict([row])

    print('Lower quantile: {}'.format(y_lower))

    clf.set_params(loss='ls')
    regression_model.fit(trainX, trainy)

    # # Make the prediction on the meshed x-axis
    y_pred = regression_model.predict([row])
    print('Predicted: {}'.format(y_pred))


 #### Predict D1_COVID_PT_CNT_M univariate

In [ ]:
from datetime import datetime
from datetime import datetime as dt, timedelta

# features_to_add =["POSITIVITY_RATE_MEAN"]
feature_to_predict = ['D1_COVID_PT_CNT_M']
lag_D1 =2
# lag_D1 =3
for f in file_list:
    df = pd.read_csv(f)
    df['SOURCE_DATA_DATE'] = df['SOURCE_DATA_DATE'].astype('datetime64[ns]')

    # # transform the time series data into supervised learning
    ## ------------------------------------------------
    index_date_sub1 = df['SOURCE_DATA_DATE'].iloc[-2]
    #datetime_object = datetime.strptime(datetime_str, '%m-%d-%y')
    # df_validation = df[df['SOURCE_DATA_DATE'] <= datetime_object ].reset_index(drop= True)
    df_validation = df[df['SOURCE_DATA_DATE'] <= index_date_sub1 ].reset_index(drop= True)
    data_multi =df_validation[feature_to_predict].values
    #data =(df_validation['COVID_MED_SURG_NO_HFNC_PCT_M'].values)/100

    # data = data.reshape(-1,1)
    data_ML = pd.DataFrame()
    for r in range(data_multi.shape[1]):
        data_current = data_multi[:,r].reshape(-1,1)
        data = series_to_supervised(data_current, n_in=lag_D1)
    #     if r != data_multi.shape[1]-1:
    #         data = data[:,:-1]

        data_ML = pd.concat([data_ML, pd.DataFrame(data)], axis= 1)

    train_current = data_ML.values
    # split into input and output columns
    # trainX, trainy = train_current[:-1, :-1], train_current[:-1, -1]
    trainX, trainy = train_current[:, :-1], train_current[:, -1]
    # # fit model

    # tscv = TimeSeriesSplit(test_size = 1, n_splits= 5,gap =1)
    tscv = TimeSeriesSplit(test_size = 1, n_splits= 30)

    alpha = 0.95
    param_grid = {'n_estimators':[50, 100, 200, 300, 500],'learning_rate':[0.01, 0.1,1],\
                                   'max_depth':[1,2,4]}
    clf = GradientBoostingRegressor(loss='quantile', alpha=alpha, random_state= 1)
    regression_model = GridSearchCV(clf, param_grid= param_grid , \
                                      cv = tscv, scoring= ['neg_mean_squared_error'],refit= 'neg_mean_squared_error', n_jobs=-1)
    regression_model.fit(trainX, trainy)

    row = trainy[-lag_D1:]
    # Make the prediction on the meshed x-axis
    y_upper = regression_model.predict([row])
    print('Upper quantile: {}'.format(y_upper))

    clf.set_params(alpha=1.0 - alpha)
    regression_model.fit(trainX, trainy)

    # Make the prediction on the meshed x-axis
    y_lower = regression_model.predict([row])

    print('Lower quantile: {}'.format(y_lower))

    clf.set_params(loss='ls')
    regression_model.fit(trainX, trainy)

    # # Make the prediction on the meshed x-axis
    y_pred = regression_model.predict([row])
    print('Predicted: {}'.format(y_pred))

####  Predict D2_COVID_PT_CNT_M with univariate analysis

In [ ]:
from datetime import datetime
from datetime import datetime as dt, timedelta
feature_to_predict = ['D2_COVID_PT_CNT_M']
lag_D2 =2
for f in file_list:
    df = pd.read_csv(f)
    #df = pd.read_csv('covid_ai_model_input_data-2021-01-26.csv')
    df['SOURCE_DATA_DATE'] = df['SOURCE_DATA_DATE'].astype('datetime64[ns]')
    # removing one row of old data
    #df[1:].reset_index(drop= True, inplace = True)

    # # transform the time series data into supervised learning
    ## ------------------------------------------------
    index_date_sub2 = df['SOURCE_DATA_DATE'].iloc[-3]
    # datetime_object = datetime.strptime(datetime_str, '%m-%d-%y')
    # df_validation = df[df['SOURCE_DATA_DATE'] <= datetime_object ].reset_index(drop= True)
    df_validation = df[df['SOURCE_DATA_DATE'] <= index_date_sub2 ].reset_index(drop= True)
    #data_multi =df_validation[features_to_add+ feature_to_predict].values
    data_multi =df_validation[feature_to_predict].values
    #data =(df_validation['COVID_MED_SURG_NO_HFNC_PCT_M'].values)/100

    # data = data.reshape(-1,1)
    data_ML = pd.DataFrame()
    for r in range(data_multi.shape[1]):
        data_current = data_multi[:,r].reshape(-1,1)
        # data = series_to_supervised(data_current, n_in=lag_D2+1)
        data = series_to_supervised(data_current, n_in=lag_D2)

    
        data_ML = pd.concat([data_ML, pd.DataFrame(data)], axis= 1)

    train_current = data_ML.values
    # split into input and output columns
    # trainX, trainy = train_current[:-1, :-1], train_current[:-1, -1]
    trainX, trainy = train_current[:, :-1], train_current[:, -1]
    # fit model

    # tscv = TimeSeriesSplit(test_size = 1, n_splits= 5,gap =1)
    tscv = TimeSeriesSplit(test_size = 1, n_splits= 30)

    alpha = 0.95
    param_grid = {'n_estimators':[50, 100, 200, 300, 500],'learning_rate':[0.01, 0.1,1],\
                                   'max_depth':[1,2,4]}
    clf = GradientBoostingRegressor(loss='quantile', alpha=alpha, random_state= 1)
    regression_model = GridSearchCV(clf, param_grid= param_grid , \
                                      cv = tscv, scoring= ['neg_mean_squared_error'],refit= 'neg_mean_squared_error', n_jobs=-1)
    regression_model.fit(trainX, trainy)

    row = trainy[-lag_D2:]
    # Make the prediction on the meshed x-axis
    y_upper = regression_model.predict([row])
    print('Upper quantile: {}'.format(y_upper))
    PTS_D2_COVID_CENSUS_PI_UPPER = y_upper[0]

    clf.set_params(alpha=1.0 - alpha)
    regression_model.fit(trainX, trainy)

    # Make the prediction on the meshed x-axis
    y_lower = regression_model.predict([row])
    print('Lower quantile: {}'.format(y_lower))
    PTS_D2_COVID_CENSUS_PI_LOWER = y_lower[0]

    clf.set_params(loss='ls')
    regression_model.fit(trainX, trainy)

    # # Make the prediction on the meshed x-axis
    y_pred = regression_model.predict([row])
    print('Predicted: {}'.format(y_pred))
    PTS_D2_COVID_CENSUS = y_pred[0]

####  Predict D2_COVID_NEW_ADM_CNT with univariate analysis

In [ ]:
from datetime import datetime
from datetime import datetime as dt, timedelta

feature_to_predict = ['D2_COVID_NEW_ADM_CNT']
lag_D2 =2

for f in ['covid_ai_model_input_data.csv']:
    df = pd.read_csv(f)
    #df = pd.read_csv('covid_ai_model_input_data-2021-01-26.csv')
    df['SOURCE_DATA_DATE'] = df['SOURCE_DATA_DATE'].astype('datetime64[ns]')
    # removing one row of old data
    #df[1:].reset_index(drop= True, inplace = True)

    # # transform the time series data into supervised learning
    ## ------------------------------------------------
    index_date_sub2 = df['SOURCE_DATA_DATE'].iloc[-3]
    # datetime_object = datetime.strptime(datetime_str, '%m-%d-%y')
    # df_validation = df[df['SOURCE_DATA_DATE'] <= datetime_object ].reset_index(drop= True)
    df_validation = df[df['SOURCE_DATA_DATE'] <= index_date_sub2 ].reset_index(drop= True)
    #data_multi =df_validation[features_to_add+ feature_to_predict].values
    data_multi =df_validation[feature_to_predict].values
    #data =(df_validation['COVID_MED_SURG_NO_HFNC_PCT_M'].values)/100

    # data = data.reshape(-1,1)
    data_ML = pd.DataFrame()
    for r in range(data_multi.shape[1]):
        data_current = data_multi[:,r].reshape(-1,1)
    #     data = series_to_supervised(data_current, n_in=lag_D2+1)
        data = series_to_supervised(data_current, n_in=lag_D2)
    #     if r == data_multi.shape[1]-1:
    #         data = data[:,1:]
    #     if r != data_test.shape[1]-1:
    #         data = data[-lag-2:,-1] ## updated to make 5 (lag of 3 +2)past days 
    #     else:
    #         data = data[-lag:,-1]

        data_ML = pd.concat([data_ML, pd.DataFrame(data)], axis= 1)

    train_current = data_ML.values
    # split into input and output columns
    # trainX, trainy = train_current[:-1, :-1], train_current[:-1, -1]
    trainX, trainy = train_current[:, :-1], train_current[:, -1]
    # # fit model

    # tscv = TimeSeriesSplit(test_size = 1, n_splits= 5,gap =1)
    tscv = TimeSeriesSplit(test_size = 1, n_splits= 30)

    # Create the current data set to make the prediction based on the trained model

    # # datetime_str = '01-09-21'
    # index_date = df['SOURCE_DATA_DATE'].iloc[-1]
    # # datetime_object = datetime.strptime(datetime_str, '%m-%d-%y')
    # # df_test = df[df['SOURCE_DATA_DATE'] <= datetime_object ].reset_index(drop= True)
    # df_test = df[df['SOURCE_DATA_DATE'] <= index_date ].reset_index(drop= True)
    # data_test =df_test[features_to_add+ feature_to_predict].values

    # data_prediction = pd.DataFrame()
    # for r in range(data_test.shape[1]):
    #     data_current = data_test[:,r].reshape(-1,1)
    #     data = series_to_supervised(data_current, n_in=lag_D2)
    # #     print(data.shape)
    #     if r != data_test.shape[1]-1:
    #         data = data[-lag_D2-2:,-1] ## updated to make 5 (lag of 3 +2)past days 
    #     else:
    #         data = data[-lag_D2:,-1]


    #     data_prediction = pd.concat([data_prediction, pd.DataFrame(data)], axis= 0)

    # # row  = data[-3:].flatten()
    # row = data_prediction.values.flatten()

    alpha = 0.95
    param_grid = {'n_estimators':[50, 100, 200, 300, 500],'learning_rate':[0.01, 0.1,1],\
                                   'max_depth':[1,2,4]}
    clf = GradientBoostingRegressor(loss='quantile', alpha=alpha, random_state= 1)
    regression_model = GridSearchCV(clf, param_grid= param_grid , \
                                      cv = tscv, scoring= ['neg_mean_squared_error'],refit= 'neg_mean_squared_error', n_jobs=-1)
    # # regression_model = GridSearchCV(clf, param_grid= param_grid , \
    # #                                 cv = tscv, scoring= ['neg_mean_squared_error'],refit= 'neg_mean_squared_error', n_jobs=-1)
    # #     regression_model = GridSearchCV(clf, param_grid= param_grid ,cv = tscv, n_jobs=-1)
    regression_model.fit(trainX, trainy)

    row = trainy[-lag_D2:]
    # Make the prediction on the meshed x-axis
    y_upper = regression_model.predict([row])
    print('Upper quantile: {}'.format(y_upper))

    clf.set_params(alpha=1.0 - alpha)
    regression_model.fit(trainX, trainy)

    # Make the prediction on the meshed x-axis
    y_lower = regression_model.predict([row])

    print('Lower quantile: {}'.format(y_lower))

    clf.set_params(loss='ls')
    regression_model.fit(trainX, trainy)

    # # Make the prediction on the meshed x-axis
    y_pred = regression_model.predict([row])
    print('Predicted: {}'.format(y_pred))



# # # # # make a one-step prediction
# # yhat = regression_model.predict(asarray([row]))

# # print('Input: %s, Predicted: %.3f' % (row, yhat[0]))

In [ ]:
row